In [1]:
# imports
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.util import random_noise
import gc

data_folder = os.path.abspath('./data')
image_folder = os.path.join(data_folder, 'TRANCOS')

print(image_folder)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available() == False:
    print('\033[91m' + "You are training on CPU, are you sure you want to continue?" + '\033[0m')

torch.manual_seed(1)

/home/alix/Desktop/school/semester_5/ECSE-415/project-415/data/TRANCOS
You are training on CPU, are you sure you want to continue?


In [2]:
# Hyperparameters
img_size = (480, 640, 3)
batch_size = 32


In [3]:
# dataset class

class CustomDataset(Dataset):
    def __init__(self, type):

        self.type = type
        csv = pd.read_csv(os.path.join(data_folder, self.type + '.csv'))

        if self.type == 'train' or self.type == 'valid':
            csv = pd.Series(csv.counts.values,index=csv.images).to_dict()

            # getting the images 
            self.images = []
            self.labels = []
            images = list(csv.keys())
            labels = list(csv.values())
            if len(images) != len(labels):
                raise ValueError("Image and label arrays do not have the same size.")

            for img, label in zip(images, labels):

                # resizing and normalizing base image
                image = cv2.normalize(cv2.resize(plt.imread(os.path.join(image_folder, img)), img_size[:2]), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                self.images.append(torch.from_numpy(image).to(device))
                self.labels.append(label)

                # data augmentation
                if self.type == 'train':
                    self.images.append(torch.from_numpy(np.fliplr(image).copy()).to(device))
                    self.images.append(torch.from_numpy(np.flipud(image).copy()).to(device))
                    for i in range(2):
                        self.labels.append(label)

            if len(self.images) != len(self.labels):
                raise ValueError("Image and label arrays do not have the same size or some values in these arrays are None.")

        elif self.type == 'test':

            # getting the images
            self.images = []
            images = list(csv.images.values.tolist())
            for img in images:

                # resizing and normalizing base image
                image = cv2.normalize(cv2.resize(plt.imread(os.path.join(image_folder, img)), img_size[:2]), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                self.images.append(torch.from_numpy(image).to(device))

        else:
            raise ValueError("Invalid type in dataset. It has to have one of the following values: 'train', 'valid', 'test'.")
        print("Initiated " + self.type + " dataset of size " + str(len(self.images)) + " and images of shape " + str(list(img_size)))
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, i):
        if self.type == 'test':
            return self.images[i]
        else:
            return self.images[i], self.labels[i]

In [4]:
# declaring datasets and data loaders

train_dataset = CustomDataset('train')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = CustomDataset('valid')
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

test_dataset = CustomDataset('test')
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Initiated train dataset of size 1869 and images of shape [480, 640, 3]
Initiated valid dataset of size 200 and images of shape [480, 640, 3]
Initiated test dataset of size 421 and images of shape [480, 640, 3]


In [5]:
# declaring parameters

In [6]:
# model

In [7]:
# training

In [8]:
# training results

In [9]:
# saving model

In [10]:
# calculating accuracy of model